In [1]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from collections import Counter

In [2]:
twitter_data = pd.read_csv('twitter_data/labels_tweets_dropna.csv')
twitter_data.head()

,Id,Cyberbullying,Insult,Profanity,Sarcasm,Threat,Exclusion,Pornography,Spam,Tweet
0,'1134395760844427264',0,0,0,0,0,0,0,0,"Seriously, I think it is a grave fault in life..."
1,'1132686520333283329',0,0,1,0,0,0,0,0,RT @feliciadetty: I hate when people stick up ...
2,'1135066181386342401',0,0,1,0,0,0,0,0,A reminder that this Catholic man is a cunt. h...
3,'1132834361945014273',0,0,0,0,0,0,0,0,RT @Headsnipe011: If people spent all their ti...
4,'1132742436185411584',0,1,1,0,0,0,0,0,i will you whore


Making overall labels of cyberbullying - this includes the labels: `cyberbullying`, `insult`, and `threats`.

In [3]:
# 0-1 labels by combining cyberbullying, insult, and threat labels
twitter_data['overall_label'] = twitter_data['Cyberbullying'] + twitter_data['Insult'] + twitter_data['Threat']
twitter_data['overall_label'] = twitter_data['overall_label'].apply(lambda x: 1 if x > 0 else 0)
twitter_data.head()

,Id,Cyberbullying,Insult,Profanity,Sarcasm,Threat,Exclusion,Pornography,Spam,Tweet,overall_label
0,'1134395760844427264',0,0,0,0,0,0,0,0,"Seriously, I think it is a grave fault in life...",0
1,'1132686520333283329',0,0,1,0,0,0,0,0,RT @feliciadetty: I hate when people stick up ...,0
2,'1135066181386342401',0,0,1,0,0,0,0,0,A reminder that this Catholic man is a cunt. h...,0
3,'1132834361945014273',0,0,0,0,0,0,0,0,RT @Headsnipe011: If people spent all their ti...,0
4,'1132742436185411584',0,1,1,0,0,0,0,0,i will you whore,1


In [4]:
twitter_data['overall_label'].value_counts()

0    19486
1     5017
Name: overall_label, dtype: int64

In [5]:
# Remove duplicate tweets
twitter_data = twitter_data.drop_duplicates(subset=['Tweet'])

In [6]:
# Data cleaning
stop_words = stopwords.words('english')
stop_words.remove('not')
stop_words.remove('no')
# remove stop words
twitter_data['Tweet'] = twitter_data['Tweet'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))
# remove punctuation
twitter_data['Tweet'] = twitter_data['Tweet'].str.replace('[^\w\s]','')
# remove numbers
twitter_data['Tweet'] = twitter_data['Tweet'].str.replace('\d+', '')
# remove extra spaces
twitter_data['Tweet'] = twitter_data['Tweet'].apply(lambda x: ' '.join(x.split()))
# remove special characters
twitter_data['Tweet'] = twitter_data['Tweet'].str.replace('[^A-Za-z0-9]+', ' ')
# all words lower case
twitter_data['Tweet'] = twitter_data['Tweet'].apply(lambda x: " ".join(x.lower() for x in x.split()))

twitter_data.head()

/var/folders/5m/yq4sh4cn2fg5plc69j8kg4sh0000gn/T/ipykernel_4231/170152066.py:8: FutureWarning: The default value of regex will change from True to False in a future version.
  twitter_data['Tweet'] = twitter_data['Tweet'].str.replace('[^\w\s]','')
/var/folders/5m/yq4sh4cn2fg5plc69j8kg4sh0000gn/T/ipykernel_4231/170152066.py:10: FutureWarning: The default value of regex will change from True to False in a future version.
  twitter_data['Tweet'] = twitter_data['Tweet'].str.replace('\d+', '')
/var/folders/5m/yq4sh4cn2fg5plc69j8kg4sh0000gn/T/ipykernel_4231/170152066.py:14: FutureWarning: The default value of regex will change from True to False in a future version.
  twitter_data['Tweet'] = twitter_data['Tweet'].str.replace('[^A-Za-z0-9]+', ' ')


,Id,Cyberbullying,Insult,Profanity,Sarcasm,Threat,Exclusion,Pornography,Spam,Tweet,overall_label
0,'1134395760844427264',0,0,0,0,0,0,0,0,seriously i think grave fault life much time w...,0
1,'1132686520333283329',0,0,1,0,0,0,0,0,rt feliciadetty i hate people stick people pie...,0
2,'1135066181386342401',0,0,1,0,0,0,0,0,a reminder catholic man cunt httpstcolsiswzcqcm,0
3,'1132834361945014273',0,0,0,0,0,0,0,0,rt headsnipe if people spent time furthering b...,0
4,'1132742436185411584',0,1,1,0,0,0,0,0,whore,1


In [7]:
twitter_data = twitter_data.drop(['Cyberbullying', 'Insult', 'Profanity', 'Sarcasm', 'Threat', 'Exclusion', 'Pornography', 'Spam'], axis=1)

In [8]:
Counter(twitter_data['overall_label'])

Counter({0: 19438, 1: 4992})

In [9]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(twitter_data, test_size=0.2)

In [10]:
overall_label_dataset_train = pd.DataFrame.to_csv(train, 'twitter_data/overall_label_dataset_train.csv', index=False)
overall_label_dataset_test = pd.DataFrame.to_csv(test, 'twitter_data/overall_label_dataset_test.csv', index=False)